# **Import necesarry modules**

In [10]:
import numpy as np

# **Functions**
## **List**
Number theoretic transform(NTT) <br>
# **Comments**
## **Number theoretic transform(NTT)**
This function is implemented by an algorithm, "Function NTT based on the Cooley-Tukey (CT) butterfly" <br>



In [38]:
class NTT:     
    def __init__(self, polynomial, q):
        #Input data: polynomial(list), q(integer)
        #Although the input data is list, we treat it as a polynomial.
        #q is modulo
        self.polynomial = polynomial
        self.degree = len(polynomial)
        self.modulo = q
        
        #Generate twiddle factors
        #Step 1) Find generator
        n = self.degree
        generator = 0 #Initial value
        test_value = 2
        while generator == 0:
            if (test_value**(2*n))%q != 1: #Does test_value is a 2n root of unity?
                test_value += 1
                continue
            else:
                pass #Yes, test_value is a 2n root of unity.
            
            test = 0 #Test for 2, 3, ..., 2n-1
            for i in range(2, 2*n):
                if (test_value**i)%q == 1:
                    test = 1 #test_value is not the 2n-primitive root of unity
                    continue
                else:
                    pass
            
            if test == 1: #For doing next test
                test_value += 1
                continue
            else:
                pass
            
            generator = test_value
        print(generator)    
        #Step 2) Gnerate psi and psi_inverse vector
        psi = []
        psi_inverse = []
        for i in range(0, n):
            psi.append((generator**i)%q)
            psi_inverse.append((generator**(2*n-i))%q)
        self.psi = psi
        self.psi_inverse = psi_inverse
        
    def introduce(self):
        #Just for testing
        print("Polynomial: ", self.polynomial)
        print("Degree: ", self.degree)
        print("Modulo: ", self.modulo)
        print("Psi: ", self.psi)
        print("Psi_inverse: ", self.psi_inverse)
        
    def CP(self):
        #Check parameters
        
        #Does degree is a power of 2?
        N = self.degree #degree
        while N > 1: 
            if N%2 == 1:
                break
                print("The degree of the given polynomial is not a power of 2.")
            else:
                N = N/2
        
        #Does modulo is suitable?
        q = self.modulo
        
        if q%int(q) != 0 or q <= 0: #Check whether q is a positive integer or not
            print("q is not a positive integer")
        else:
            pass
        
        if q%(2*self.degree) == 1: #Check q (mod 2n) = 1
            pass
        else:
            print("q mod 2n != 1")
        
        if q == 1: #Check whether q is a prime number or not
            print("modulo is not a prime number")
        elif q > 1:
            for i in range(2, q):
                if q%i == 0:
                    break
                    print("q is not a prime number")
                else:
                    pass   
    
    def encode(self):
        #Pointwise multiplication between polynomial and psi
        encoded = []
        polynomial = self.polynomial
        psi = self.psi
        for i in range(0, n):
            encoded.append(polynomial[i]*psi[i])
            
        return encoded
    
    def ntt(self):
        n = self.degree
        t = self.degree
        m = 1
        while m < n:
            t = t/2
            for i in range(0, m):
                j_1 = 2*i*t
                j_2 = j_1 + t - 1
                s = self
    
        

In [39]:
a = NTT([0,1,2,3,4,5,6,7], 17)
a.introduce()
a.CP()

3
Polynomial:  [0, 1, 2, 3, 4, 5, 6, 7]
Degree:  8
Modulo:  17
Psi:  [1, 3, 9, 10, 13, 5, 15, 11]
Psi_inverse:  [1, 6, 2, 12, 4, 7, 8, 14]
